In [1]:
using Flux
using Flux: Data.DataLoader
using Flux: onehotbatch,onecold,crossentropy
using Flux: @epochs
using Statistics
using MLDatasets

In [12]:
x_train_temp,y_train_temp = MNIST.traindata();
x_test_temp,y_test_temp = MNIST.testdata();

In [13]:
println("The size of the training images are $(size(x_train_temp)) and the size of training labels is $(size(y_train_temp))")
print("The size of the testing images are $(size(x_test_temp)) and the size of testing labels is $(size(y_test_temp))")

The size of the training images are (28, 28, 60000) and the size of training labels is (60000,)
The size of the testing images are (28, 28, 10000) and the size of testing labels is (10000,)

In [16]:
size(y_train_temp)

(60000,)

In [18]:
x_train = Flux.unsqueeze(x_train_temp[:,:,1:100],3)
x_test = Flux.unsqueeze(x_test_temp[:,:,1:100],3)

y_train = onehotbatch(y_train_temp[1:100],0:9)
y_test = onehotbatch(y_test_temp[1:100],0:9)

#Displaying the updated shape of the elements

println("The size of the training images are $(size(x_train)) and the size of training labels is $(size(y_train))")
print("The size of the testing images are $(size(x_test)) and the size of testing labels is $(size(y_test))")

train_data = DataLoader(x_train, y_train, batchsize=8);

The size of the training images are (28, 28, 1, 100) and the size of training labels is (10, 100)
The size of the testing images are (28, 28, 1, 100) and the size of testing labels is (10, 100)

In [19]:
layer1 = Conv((3,3),1=>8,relu,stride=2)
layer2 = Conv((3,3),8=>16,relu)
layer3 = Conv((3,3),16=>32,relu)
layer4 = GlobalMaxPool()
layer5 = flatten
layer6 = Dense(32,10)
layer7 = softmax;

In [20]:
layers = [layer1, layer2, layer3, layer4, layer5, layer6, layer7]
x = x_train

for l in layers
    println("The layer is $l")
    println("The size of the input layer is $(size(x))")
    x = l(x)
    println("The size of the output layer is $(size(x))")
    println("-"^30)
end

The layer is Conv((3, 3), 1=>8, relu)
The size of the input layer is (28, 28, 1, 100)
The size of the output layer is (13, 13, 8, 100)
------------------------------
The layer is Conv((3, 3), 8=>16, relu)
The size of the input layer is (13, 13, 8, 100)
The size of the output layer is (11, 11, 16, 100)
------------------------------
The layer is Conv((3, 3), 16=>32, relu)
The size of the input layer is (11, 11, 16, 100)
The size of the output layer is (9, 9, 32, 100)
------------------------------
The layer is GlobalMaxPool()
The size of the input layer is (9, 9, 32, 100)
The size of the output layer is (1, 1, 32, 100)
------------------------------
The layer is flatten
The size of the input layer is (1, 1, 32, 100)
The size of the output layer is (32, 100)
------------------------------
The layer is Dense(32, 10)
The size of the input layer is (32, 100)
The size of the output layer is (10, 100)
------------------------------
The layer is softmax
The size of the input layer is (10, 100)

In [40]:
# model = Chain(layer1, layer2, layer3, layer4, layer5,layer6, layer7)
# # model = Chain(Dense(784,32),Dense(32,10),softmax)
# L(x,y) = Flux.crossentropy(model(x),y)
# opt = Flux.Optimise.ADAM()
# ps = Flux.params(model)

# evalcb() = @show(L(x_train, y_train))
# throttled_cb = Flux.throttle(evalcb, 1)

# # callback() = push!(loss_vector, L(x_train, y_train))
# # loss_vector = Vector{Float32}()
# print("Training started")

# @epochs 100 Flux.train!(L,ps,train_data,opt, cb = throttled_cb)

Training startedL(x_train, y_train) = 0.00027767473f0


┌ Info: Epoch 1
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 2
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 3
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 4
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 5
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 6
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 7
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 8
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 9
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 10
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 11
└ 

L(x_train, y_train) = 2.3973078f-6


┌ Info: Epoch 13
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 14
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 15
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 16
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 17
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 18
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 19
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 20
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 21
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 22
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Ep

L(x_train, y_train) = 1.0919591f-6


...

L(x_train, y_train) = 1.5735634f-7


┌ Info: Epoch 77
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 78
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 79
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 80
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 81
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 82
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 83
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 84
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 85
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 86
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Ep

L(x_train, y_train) = 9.536747f-8


┌ Info: Epoch 88
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 89
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 90
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 91
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 92
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 93
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 94
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 95
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 96
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 97
└ @ Main C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Ep

L(x_train, y_train) = 5.7220486f-8


In [43]:
model1 = Chain(layer1, layer2, layer3, layer4, layer5,layer6, layer7)
L1(x,y) = Flux.crossentropy(model1(x),y)
opt1 = Flux.Optimise.ADAM()
ps1 = Flux.params(model1)

total_step = 100

for step in 1:total_step
    Flux.train!(L1,ps1,train_data,opt1)
    println("Training Loss is $(L1(x_train,y_train))")
    println("Validation Loss is $(L1(x_test,y_test))")
    println("-"^50)
end

Training Loss is 4.088891e-7
Validation Loss is 2.1624796
--------------------------------------------------
Training Loss is 1.5139618e-7
Validation Loss is 2.1598842
--------------------------------------------------
Training Loss is 1.031191e-6
Validation Loss is 2.1051345
--------------------------------------------------
Training Loss is 5.304881e-5
Validation Loss is 2.8357918
--------------------------------------------------
Training Loss is 2.3961195e-7
Validation Loss is 2.1637197
--------------------------------------------------
Training Loss is 1.0132831e-7
Validation Loss is 2.0893497
--------------------------------------------------
Training Loss is 2.6226159e-8
Validation Loss is 2.0802162
--------------------------------------------------
Training Loss is -4.2915318e-8
Validation Loss is 2.064498
--------------------------------------------------
Training Loss is -6.675719e-8
Validation Loss is 2.0647378
--------------------------------------------------
Training Loss

Training Loss is -1.1682509e-7
Validation Loss is 2.2350347
--------------------------------------------------
Training Loss is -1.15633e-7
Validation Loss is 2.2366495
--------------------------------------------------
Training Loss is -1.15633e-7
Validation Loss is 2.235707
--------------------------------------------------
Training Loss is -1.1801719e-7
Validation Loss is 2.2399514
--------------------------------------------------
Training Loss is -1.1444091e-7
Validation Loss is 2.2426443
--------------------------------------------------
Training Loss is -1.15633e-7
Validation Loss is 2.2428238
--------------------------------------------------
Training Loss is -1.1801719e-7
Validation Loss is 2.240778
--------------------------------------------------
Training Loss is -1.15633e-7
Validation Loss is 2.2492137
--------------------------------------------------
Training Loss is -1.15633e-7
Validation Loss is 2.245593
--------------------------------------------------
Training Loss 

In [38]:
accuracy(model(x_test),y_test)

0.71

In [114]:
foo=reshape(x_train_temp,(784,60000));

In [117]:
size(x_train_temp)

(28, 28, 60000)

In [120]:
preprocess(p) = vec(Float64.(p))
foo = preprocess.(x_train_temp[1:100])

LoadError: MethodError: no method matching vec(::Float64)
Closest candidates are:
  vec(!Matched::LinearAlgebra.Transpose{T,var"#s828"} where var"#s828"<:(AbstractArray{T,1} where T) where T) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\LinearAlgebra\src\adjtrans.jl:226
  vec(!Matched::LinearAlgebra.Adjoint{var"#s828",var"#s8281"} where var"#s8281"<:(AbstractArray{T,1} where T) where var"#s828"<:Real) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\LinearAlgebra\src\adjtrans.jl:227
  vec(!Matched::SparseArrays.AbstractSparseArray{Tv,Ti,1} where Ti where Tv) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\SparseArrays\src\sparsevector.jl:911
  ...

In [126]:
length.(x_train_temp)

28×28×60000 Array{Int64,3}:
[:, :, 1] =
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1 